*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

zsh:1: command not found: pip
zsh:1: command not found: pip
2.7.0


In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence

In [15]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

zsh:1: command not found: wget
zsh:1: command not found: wget


In [43]:
train = pd.read_csv(train_file_path, sep="\t", header=None, names=["ham_or_spam", "msg"])
train.dropna()
train.shape
train.tail()

,ham_or_spam,msg
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...
4178,ham,tell my bad character which u dnt lik in me. ...


In [44]:
test = pd.read_csv(test_file_path, sep="\t", header=None, names=["ham_or_spam", "msg"])
test.dropna()
test.shape

(1392, 2)

In [45]:
#make labels 1s and 0s for training
train["ham_or_spam"] = pd.factorize(train["ham_or_spam"])[0]
test["ham_or_spam"] = pd.factorize(test["ham_or_spam"])[0]

In [46]:
train.tail()

,ham_or_spam,msg
4174,0,just woke up. yeesh its late. but i didn't fal...
4175,0,what do u reckon as need 2 arrange transport i...
4176,1,free entry into our £250 weekly competition ju...
4177,1,-pls stop bootydelious (32/f) is inviting you ...
4178,0,tell my bad character which u dnt lik in me. ...


In [47]:
train_labels =  train["ham_or_spam"].values
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train["msg"].values, train_labels)
)
train_dataset

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [48]:
test_labels =  test["ham_or_spam"].values
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test["msg"].values, test_labels)
)
test_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [49]:
#Next shuffle the data for training and create batches of these (text, label) pairs:
BUFFER_SIZE = 100
BATCH_SIZE = 32

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [50]:
#Create a TextVectorization layer for our model
#https://www.tensorflow.org/text/tutorials/text_classification_rnn
vec = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

vec.adapt(train_dataset.map(lambda text, label: text))

In [51]:
vocab = np.array(vec.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in', 'is',
       'me', 'my', 'for', 'your', 'of', 'it', 'call', 'have', 'on'],
      dtype='<U15')

In [52]:
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])

In [53]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [54]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
131/131 [==============================] - 569s 4s/step - loss: 0.6032 - accuracy: 0.8660 - val_loss: 0.4878 - val_accuracy: 0.8604
Epoch 2/10
131/131 [==============================] - 514s 4s/step - loss: 0.3659 - accuracy: 0.8660 - val_loss: 0.2171 - val_accuracy: 0.8604
Epoch 3/10
131/131 [==============================] - 318s 2s/step - loss: 0.1492 - accuracy: 0.9423 - val_loss: 0.0976 - val_accuracy: 0.9750
Epoch 4/10
131/131 [==============================] - 322s 2s/step - loss: 0.0856 - accuracy: 0.9806 - val_loss: 0.0746 - val_accuracy: 0.9865
Epoch 5/10
131/131 [==============================] - 315s 2s/step - loss: 0.0650 - accuracy: 0.9856 - val_loss: 0.0668 - val_accuracy: 0.9865
Epoch 6/10
131/131 [==============================] - 312s 2s/step - loss: 0.0531 - accuracy: 0.9880 - val_loss: 0.0623 - val_accuracy: 0.9875
Epoch 7/10
131/131 [==============================] - 317s 2s/step - loss: 0.0439 - accuracy: 0.9897 - val_loss: 0.0589 - val_accuracy: 0.9885

In [55]:
test_loss, test_acc = model.evaluate(test_dataset)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

44/44 [==============================] - 20s 442ms/step - loss: 0.0604 - accuracy: 0.9856
Test Loss: 0.060394950211048126
Test Accuracy: 0.9856321811676025


In [56]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    prediction = model.predict([pred_text])
    print(prediction)
    p = prediction[0][0]
    return [p, "ham" if p <0.5 else "spam"]

pred_text = "how are you doing today?"

prediction1 = predict_message(pred_text)
print(prediction1)

[[-6.694447]]
[-6.694447, 'ham']


In [57]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[[-6.694447]]
[[2.1711726]]
[[-10.947942]]
[[5.22707]]
[[6.36064]]
[[-9.512832]]
[[-9.987446]]
You passed the challenge. Great job!
